# SiPhON Phase 0.1: Analytical Physics Baseline

This notebook establishes the theoretical foundation for silicon photonic ring resonator analysis using closed-form analytical equations.

## Objectives
1. Implement and validate the all-pass ring resonator transfer function
2. Extract key metrics: FSR, Q, ER, Finesse
3. Model thermo-optic tuning and heater power budget
4. Compare results against literature values

## Theory

### All-Pass Ring Transfer Function

The transmission through an all-pass ring resonator is:

$$T(\phi) = \frac{a^2 - 2ra\cos(\phi) + r^2}{1 - 2ra\cos(\phi) + (ra)^2}$$

where:
- $a = e^{-\alpha L/2}$ is the round-trip amplitude transmission
- $r = \sqrt{1-\kappa^2}$ is the self-coupling coefficient
- $\phi = \beta L = \frac{2\pi n_{eff}}{\lambda} \cdot 2\pi R$ is the round-trip phase

### Key Metrics

- **Free Spectral Range (FSR):** $\text{FSR} = \frac{\lambda^2}{n_g \cdot L}$
- **Quality Factor:** $Q = \frac{\lambda}{\Delta\lambda}$
- **Finesse:** $F = \frac{\text{FSR}}{\Delta\lambda}$

In [ ]:
# Setup and imports
import sys
from pathlib import Path

# Add siphon to path (for development)
sys.path.insert(0, str(Path.cwd().parent / 'python'))

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import EngFormatter

# SiPhON modules
from siphon.ring import RingResonator, RingGeometry
from siphon.thermal import ThermalModel, ThermalConfig

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14

## 1. Ring Resonator Definition

We define a typical silicon ring resonator with parameters representative of foundry-fabricated devices.

In [ ]:
# Define typical silicon ring resonator
geometry = RingGeometry(
    radius=10e-6,       # 10 μm radius
    kappa=0.2,          # 20% power coupling
    alpha=2.0,          # 2 dB/cm propagation loss
    n_eff=2.4,          # Effective index (TE mode)
    n_g=4.2,            # Group index
)

ring = RingResonator(geometry)
print(ring)

# Display geometry parameters
print(f"\nGeometry Parameters:")
print(f"  Circumference L = {geometry.circumference * 1e6:.2f} μm")
print(f"  Self-coupling r = {geometry.self_coupling:.4f}")
print(f"  Round-trip loss a = {geometry.round_trip_loss:.4f}")

## 2. Transmission Spectrum

Generate the transmission spectrum showing periodic resonances.

In [ ]:
# Operating wavelength
wavelength_center = 1.55e-6  # 1550 nm (C-band)

# Generate spectrum over 3 FSRs
fsr = ring.fsr(wavelength_center)
wavelength_span = 3 * fsr

wavelengths, transmission = ring.spectrum(
    wavelength_center=wavelength_center,
    wavelength_span=wavelength_span,
    n_points=2000
)

# Plot transmission spectrum
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Linear scale
ax1.plot((wavelengths - wavelength_center) * 1e9, transmission, 'b-', lw=1.5)
ax1.set_xlabel('Wavelength detuning (nm)')
ax1.set_ylabel('Transmission')
ax1.set_title('Ring Resonator Transmission Spectrum (Linear Scale)')
ax1.set_ylim(0, 1.05)
ax1.axhline(y=1, color='gray', linestyle='--', alpha=0.5)

# Mark FSR
ax1.annotate('', xy=(fsr*1e9/2, 0.5), xytext=(-fsr*1e9/2, 0.5),
             arrowprops=dict(arrowstyle='<->', color='red', lw=2))
ax1.text(0, 0.55, f'FSR = {fsr*1e9:.2f} nm', ha='center', fontsize=11, color='red')

# dB scale
transmission_db = 10 * np.log10(transmission)
ax2.plot((wavelengths - wavelength_center) * 1e9, transmission_db, 'b-', lw=1.5)
ax2.set_xlabel('Wavelength detuning (nm)')
ax2.set_ylabel('Transmission (dB)')
ax2.set_title('Ring Resonator Transmission Spectrum (Log Scale)')
ax2.set_ylim(-25, 2)

plt.tight_layout()
plt.show()

## 3. Key Metrics Extraction

Extract and display all key performance metrics.

In [ ]:
# Extract all metrics
metrics = ring.metrics(wavelength=wavelength_center)

print("=" * 50)
print("Ring Resonator Performance Metrics")
print("=" * 50)
print(f"\nSpectral Metrics:")
print(f"  Free Spectral Range (FSR) = {metrics.fsr * 1e9:.3f} nm")
print(f"  Free Spectral Range (FSR) = {metrics.fsr_ghz:.1f} GHz")
print(f"  Linewidth (FWHM) Δλ = {metrics.linewidth * 1e12:.2f} pm")
print(f"\nQuality Metrics:")
print(f"  Quality Factor Q = {metrics.quality_factor:.0f}")
print(f"  Finesse F = {metrics.finesse:.1f}")
print(f"  Extinction Ratio ER = {metrics.extinction_ratio_db:.1f} dB")

## 4. Parameter Sensitivity Studies

### 4.1 FSR vs. Radius

Verify that FSR decreases with increasing radius as expected from $\text{FSR} = \lambda^2 / (n_g \cdot 2\pi R)$.

In [ ]:
# FSR vs Radius
radii = np.linspace(5e-6, 50e-6, 50)  # 5-50 μm
fsr_values = []

for R in radii:
    geom = RingGeometry(radius=R, kappa=0.2, alpha=2.0, n_eff=2.4, n_g=4.2)
    ring_temp = RingResonator(geom)
    fsr_values.append(ring_temp.fsr(wavelength_center))

fsr_values = np.array(fsr_values)

# Analytical formula for comparison
fsr_analytical = wavelength_center**2 / (4.2 * 2 * np.pi * radii)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(radii * 1e6, fsr_values * 1e9, 'b-', lw=2, label='SiPhON Model')
ax.plot(radii * 1e6, fsr_analytical * 1e9, 'r--', lw=2, label='Analytical: λ²/(n_g·2πR)')
ax.set_xlabel('Ring Radius (μm)')
ax.set_ylabel('FSR (nm)')
ax.set_title('Free Spectral Range vs. Ring Radius')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Verify match
max_error = np.max(np.abs(fsr_values - fsr_analytical) / fsr_analytical) * 100
print(f"Maximum relative error vs. analytical formula: {max_error:.6f}%")

### 4.2 Quality Factor vs. Loss

Verify that Q increases with decreasing propagation loss.

In [ ]:
# Q vs Loss (multiple coupling values)
loss_values = np.linspace(0.5, 10, 50)  # 0.5-10 dB/cm
kappa_values = [0.1, 0.2, 0.3, 0.5]

fig, ax = plt.subplots(figsize=(10, 6))

for kappa in kappa_values:
    Q_values = []
    for alpha in loss_values:
        geom = RingGeometry(radius=10e-6, kappa=kappa, alpha=alpha, n_eff=2.4, n_g=4.2)
        ring_temp = RingResonator(geom)
        Q_values.append(ring_temp.quality_factor(wavelength_center))
    
    ax.semilogy(loss_values, Q_values, lw=2, label=f'κ = {kappa}')

ax.set_xlabel('Propagation Loss (dB/cm)')
ax.set_ylabel('Quality Factor Q')
ax.set_title('Quality Factor vs. Propagation Loss')
ax.legend(title='Coupling')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 4.3 Extinction Ratio vs. Coupling

Explore the relationship between coupling coefficient and extinction ratio, identifying the critical coupling condition.

In [ ]:
# ER vs Coupling (multiple loss values)
kappa_sweep = np.linspace(0.05, 0.8, 100)
alpha_values = [1.0, 2.0, 5.0]

fig, ax = plt.subplots(figsize=(10, 6))

for alpha in alpha_values:
    ER_values = []
    for kappa in kappa_sweep:
        geom = RingGeometry(radius=10e-6, kappa=kappa, alpha=alpha, n_eff=2.4, n_g=4.2)
        ring_temp = RingResonator(geom)
        ER_values.append(ring_temp.extinction_ratio(wavelength_center))
    
    ax.plot(kappa_sweep, ER_values, lw=2, label=f'α = {alpha} dB/cm')
    
    # Find and mark critical coupling (max ER)
    max_idx = np.argmax(ER_values)
    ax.axvline(x=kappa_sweep[max_idx], linestyle=':', alpha=0.5)

ax.set_xlabel('Power Coupling Coefficient κ')
ax.set_ylabel('Extinction Ratio (dB)')
ax.set_title('Extinction Ratio vs. Coupling (Critical Coupling = ER Maximum)')
ax.legend(title='Loss')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Thermal Tuning Model

Model the thermo-optic effect for thermal tuning of the ring resonator.

In [ ]:
# Create thermal model with default silicon parameters
thermal = ThermalModel(ring)

# Get thermal metrics
thermal_metrics = thermal.metrics(wavelength=wavelength_center)

print("=" * 50)
print("Thermal Tuning Parameters")
print("=" * 50)
print(f"\nWavelength Shift:")
print(f"  Δλ/ΔT = {thermal_metrics.wavelength_shift_per_kelvin * 1e12:.2f} pm/K")
print(f"\nFSR Tuning:")
print(f"  Temperature per FSR = {thermal_metrics.temperature_per_fsr:.1f} K")
print(f"  Power per FSR = {thermal_metrics.power_per_fsr * 1e3:.2f} mW")
print(f"\nTuning Efficiency:")
print(f"  Efficiency = {thermal_metrics.tuning_efficiency * 1e12:.1f} pm/mW")
print(f"  Max tuning range = {thermal_metrics.max_tuning_range * 1e9:.2f} nm (@ {thermal.config.max_heater_power*1e3:.0f} mW)")

### 5.1 Thermally Shifted Spectrum

Visualize how the resonance shifts with temperature.

In [ ]:
# Generate spectra at different temperatures
temperatures = [0, 10, 20, 30]  # K above ambient
colors = plt.cm.hot(np.linspace(0.3, 0.8, len(temperatures)))

fig, ax = plt.subplots(figsize=(12, 6))

for delta_T, color in zip(temperatures, colors):
    if delta_T == 0:
        wl, T = ring.spectrum(wavelength_center, n_points=2000)
        label = 'Ambient (300 K)'
    else:
        wl, T = thermal.shifted_spectrum(wavelength_center, delta_T, n_points=2000)
        label = f'+{delta_T} K'
    
    ax.plot((wl - wavelength_center) * 1e9, T, color=color, lw=1.5, label=label)

ax.set_xlabel('Wavelength detuning from 1550 nm (nm)')
ax.set_ylabel('Transmission')
ax.set_title('Thermal Tuning of Ring Resonance')
ax.legend(title='Temperature')
ax.set_ylim(0, 1.05)
ax.grid(True, alpha=0.3)

# Add arrow showing red-shift direction
ax.annotate('Heating\n(Red-shift)', xy=(5, 0.3), fontsize=10, ha='center')
ax.annotate('', xy=(8, 0.2), xytext=(2, 0.2),
             arrowprops=dict(arrowstyle='->', color='red', lw=2))

plt.tight_layout()
plt.show()

### 5.2 Heater Power Budget

Calculate the heater power required to tune across the full FSR.

In [ ]:
# Power budget analysis for different radii
radii = np.array([5, 10, 20, 30, 50]) * 1e-6

print("Heater Power Budget Analysis")
print("=" * 60)
print(f"{'Radius (μm)':<12} {'FSR (nm)':<12} {'ΔT/FSR (K)':<12} {'P/FSR (mW)':<12}")
print("-" * 60)

power_per_fsr_values = []
fsr_nm_values = []

for R in radii:
    geom = RingGeometry(radius=R, kappa=0.2, alpha=2.0, n_eff=2.4, n_g=4.2)
    ring_temp = RingResonator(geom)
    thermal_temp = ThermalModel(ring_temp)
    
    fsr_nm = ring_temp.fsr(wavelength_center) * 1e9
    temp_fsr = thermal_temp.temperature_per_fsr(wavelength_center)
    power_fsr = thermal_temp.power_per_fsr(wavelength_center) * 1e3  # mW
    
    fsr_nm_values.append(fsr_nm)
    power_per_fsr_values.append(power_fsr)
    
    print(f"{R*1e6:<12.0f} {fsr_nm:<12.2f} {temp_fsr:<12.1f} {power_fsr:<12.2f}")

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.plot(radii * 1e6, fsr_nm_values, 'bo-', lw=2, markersize=8)
ax1.set_xlabel('Ring Radius (μm)')
ax1.set_ylabel('FSR (nm)')
ax1.set_title('FSR vs. Radius')
ax1.grid(True, alpha=0.3)

ax2.plot(radii * 1e6, power_per_fsr_values, 'ro-', lw=2, markersize=8)
ax2.set_xlabel('Ring Radius (μm)')
ax2.set_ylabel('Power per FSR (mW)')
ax2.set_title('Heater Power Budget vs. Radius')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Literature Validation

Compare our results against expected values from silicon photonics literature.

In [ ]:
print("=" * 60)
print("Literature Validation")
print("=" * 60)

# Test case: 10 μm radius ring at 1550 nm
test_geom = RingGeometry(radius=10e-6, kappa=0.2, alpha=2.0, n_eff=2.4, n_g=4.2)
test_ring = RingResonator(test_geom)
test_thermal = ThermalModel(test_ring)

# Expected values from literature
expected = {
    'FSR': (8, 10),           # nm, for R=10μm, n_g=4.2
    'dlambda_dT': (0.06, 0.12),  # nm/K for silicon at 1550nm
}

# Computed values
fsr_computed = test_ring.fsr(wavelength_center) * 1e9  # nm
dlambda_dT_computed = test_thermal.wavelength_shift_per_kelvin(wavelength_center) * 1e9  # nm/K

print(f"\nFSR (R=10μm):")
print(f"  Computed: {fsr_computed:.2f} nm")
print(f"  Expected: {expected['FSR'][0]}-{expected['FSR'][1]} nm")
fsr_pass = expected['FSR'][0] <= fsr_computed <= expected['FSR'][1]
print(f"  Status: {'✓ PASS' if fsr_pass else '✗ FAIL'}")

print(f"\nΔλ/ΔT (silicon at 1550nm):")
print(f"  Computed: {dlambda_dT_computed:.3f} nm/K")
print(f"  Expected: {expected['dlambda_dT'][0]}-{expected['dlambda_dT'][1]} nm/K")
thermal_pass = expected['dlambda_dT'][0] <= dlambda_dT_computed <= expected['dlambda_dT'][1]
print(f"  Status: {'✓ PASS' if thermal_pass else '✗ FAIL'}")

print("\n" + "=" * 60)
if fsr_pass and thermal_pass:
    print("All validation checks PASSED")
else:
    print("Some validation checks FAILED - review parameters")
print("=" * 60)

## 7. Summary

### Analytical Baseline Established

This notebook demonstrates the core analytical models for Phase 0.1:

1. **Ring Transfer Function**: All-pass transmission $T(\lambda)$ correctly shows periodic resonances with FSR, Q, and ER matching analytical predictions.

2. **Key Metrics**: 
   - FSR scales as $\lambda^2/(n_g \cdot L)$ ✓
   - Q increases with decreasing loss ✓
   - ER maximized at critical coupling ✓

3. **Thermal Model**:
   - $\Delta\lambda/\Delta T \approx 0.08-0.1$ nm/K (matches silicon literature) ✓
   - Heater power budget defined for FSR tuning ✓

### Next Steps (Phase 0.2)

- Define fabrication tolerance distributions ($\sigma_w$, $\sigma_h$)
- Implement Monte Carlo variability engine
- Calculate yield based on thermal power budget

In [ ]:
# Final summary table
print("\n" + "=" * 70)
print("PHASE 0.1 ANALYTICAL BASELINE - SUMMARY")
print("=" * 70)
print(f"\nReference Device: R = 10 μm, κ = 0.2, α = 2 dB/cm")
print(f"Operating Wavelength: {wavelength_center*1e9:.0f} nm\n")

m = ring.metrics(wavelength_center)
tm = thermal.metrics(wavelength_center)

print(f"{'Metric':<30} {'Value':<20} {'Unit'}")
print("-" * 70)
print(f"{'Free Spectral Range':<30} {m.fsr*1e9:<20.3f} nm")
print(f"{'Quality Factor':<30} {m.quality_factor:<20.0f} -")
print(f"{'Finesse':<30} {m.finesse:<20.1f} -")
print(f"{'Extinction Ratio':<30} {m.extinction_ratio_db:<20.1f} dB")
print(f"{'Linewidth (FWHM)':<30} {m.linewidth*1e12:<20.2f} pm")
print("-" * 70)
print(f"{'Thermal Shift (Δλ/ΔT)':<30} {tm.wavelength_shift_per_kelvin*1e12:<20.2f} pm/K")
print(f"{'Temperature per FSR':<30} {tm.temperature_per_fsr:<20.1f} K")
print(f"{'Power per FSR':<30} {tm.power_per_fsr*1e3:<20.2f} mW")
print(f"{'Tuning Efficiency':<30} {tm.tuning_efficiency*1e12:<20.1f} pm/mW")
print("=" * 70)